In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os

tickers = pd.read_excel(os.path.join(os.getcwd(),'Tickers.xlsx'))['Ticker Symbol']
# Define conditions for selecting stocks
current_ratio = 1.5
interest_coverage = 3
eps = 2
sectors = ['Consumer Staples', 'Healthcare', 'Utilities']

# Retrieve list of all publicly traded companies
all_tickers = yf.Tickers('')

# Filter stocks based on conditions
selected_tickers = []
for ticker in all_tickers.tickers:
    info = ticker.info
    if info['sector'] in sectors and \
    info['currentRatio'] > current_ratio and \
    info['interestCoverage'] > interest_coverage and \
    info['eps'] > eps:
    selected_tickers.append(info['symbol'])

# Get financial data for selected tickers
start_date = dt.datetime.now() - dt.timedelta(days=365*5)
end_date = dt.datetime.now()
data = yf.download(selected_tickers, start=start_date, end=end_date)

# Calculate expected returns of each stock using CAPM
risk_free_rate = 0.02
market_risk_premium = 0.08
beta = data['Adj Close'].pct_change().corr(data['Adj Close']['SPY'].pct_change())
expected_returns = risk_free_rate + beta * market_risk_premium

# Calculate expected return and volatility of portfolio
weights = np.array([1/len(selected_tickers)] * len(selected_tickers))
expected_return = np.sum(expected_returns * weights)
portfolio_variance = np.dot(weights.T, np.dot(data['Adj Close'].pct_change().cov(), weights))
portfolio_volatility = np.sqrt(portfolio_variance) * np.sqrt(252)

# Optimize portfolio to minimize volatility while maintaining expected return
def portfolio_volatility(weights):
return np.sqrt(np.dot(weights.T, np.dot(data['Adj Close'].pct_change().cov(), weights))) * np.sqrt(252)

def portfolio_return(weights):
return np.sum(expected_returns * weights)

def constraint(weights):
return portfolio_return(weights) - expected_return

bounds = [(0, 1)] * len(selected_tickers)
constraints = ({'type': 'eq', 'fun': constraint})
optimized = minimize(portfolio_volatility, weights, method='SLSQP', bounds=bounds, constraints=constraints)

# Print optimized portfolio
print("Optimized Portfolio:")
for i in range(len(selected_tickers)):
print(selected_tickers[i], optimized.x[i])

# Visualize portfolio
fig, ax = plt.subplots()
ax.scatter(np.sqrt(np.diag(data['Adj Close'].pct_change().cov()))*np.sqrt(252), expected_returns, c='b', s=50)
ax.scatter(portfolio_volatility, expected_return, c='r', s=200)
for i in range(len(selected_tickers)):
ax.annotate(selected_tickers[i], (np.sqrt(np.diag(data['Adj Close'].pct_change().cov()))[i]*np.sqrt(252), expected_returns[i]))
plt.xlabel('Volatility (annualized)')
plt.ylabel('Expected Return')
plt.title('Portfolio Optimization')
plt.show()

IndentationError: expected an indented block (3867734300.py, line 23)

In [4]:
import pandas as pd
import os
tickers = pd.read_excel(os.path.join(os.getcwd(),'Tickers.xlsx'))['Ticker Symbol']
tickers

0           A
1          AA
2         AAC
3         AAL
4        AAMC
        ...  
5604     ZWRK
5605    ZWRKU
5606       ZY
5607     ZYNE
5608     ZYXI
Name: Ticker Symbol, Length: 5609, dtype: object

In [ ]:
!pip uninstall numpy
!pip install numpy==1.17.1

Found existing installation: numpy 1.16.6
Uninstalling numpy-1.16.6:
  Would remove:
    /Users/zaid/anaconda3/envs/Beststocke/bin/f2py
    /Users/zaid/anaconda3/envs/Beststocke/bin/f2py3
    /Users/zaid/anaconda3/envs/Beststocke/bin/f2py3.9
    /Users/zaid/anaconda3/envs/Beststocke/lib/python3.9/site-packages/numpy
    /Users/zaid/anaconda3/envs/Beststocke/lib/python3.9/site-packages/numpy-1.16.6-py3.9.egg-info
Proceed (Y/n)? 